**LeNet-5 Modified Without Batch Normalization or Dropout Layer**
---



In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(16,120,5),
            nn.ReLU(),
            nn.Flatten())

        self.fc = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
            nn.LogSoftmax(dim=-1))

        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        y = self.convnet(x)
        out = self.fc(y)
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        outputs = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.306809
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.291311
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.227425
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.136307
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.015715
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.018222
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.008607
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.954003
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.912950
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.082973
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.819596
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.907707
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.746715
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.790530
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.882849
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.858795
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.740097
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.893781
Train Epoch: 1 [23

**LeNet-5 Modified With Dropout of 0.25**

In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.Dropout(0.25) #Dropout Layer
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(16,120,5),
            nn.ReLU(),
            nn.Flatten())

        self.fc = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
            nn.LogSoftmax(dim=-1))

        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        y = self.convnet(x)
        out = self.fc(y)
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        outputs = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.305384
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.287058
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.192961
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.114276
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.090188
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.928884
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.969677
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.046214
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.888032
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.840638
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.882881
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.991352
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.758903
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.884164
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.857030
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.818795
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.702539
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.940417
Train Epoch: 1 [23

**LeNet-5 Modified with Batch Normalization**

In [8]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.BatchNorm2d(6), #Batch Normalization Layer
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(16,120,5),
            nn.ReLU(),
            nn.Flatten())

        self.fc = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
            nn.LogSoftmax(dim=-1))

        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        y = self.convnet(x)
        
        out = self.fc(y)
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        outputs = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.308962
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.292907
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.227754
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.298429
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.106616
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.082714
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.969209
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.111538
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.895650
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.896650
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.831419
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.820726
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.737247
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.861948
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.874434
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.837155
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.783866
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.032804
Train Epoch: 1 [23

The network without a drop out layer or batch normalization performed the worst with a test accuracy of 52% over 5 epochs. The network with a dropout layer of 0.25 performed the second best with a test accuracy of 54%. The network with batch normalization performed the best with an accuracy of 56% over 5 epochs. As far as the training time is concerned, the plain network was the fastest with a training time of 171 seconds. The network with the dropout layer was the second slowest with a training time of 180 seconds. The network with the batch normalization was the slowest with a training time of 185 seconds. Screenshots of the testing loss are provided. If you wish to see training loss, expand the raw output data in this file.